These are some old examples.
There has been some refactoring so that it is easier to use now.
The examples are still around as they shows some lower level features.

## Setup

All the imports.
First, adding the additional repo for the dependecies and then loading `libgcode`.
Don't forget to run `sbt publishLocal` in the `libgcode` folder.

In [ ]:
import coursierapi._
interp.repositories() +:= MavenRepository.of("https://jitpack.io")

In [ ]:
import $ivy.`com.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode.utils.Viewer
import libgcode._
import libgcode.extractor._
import java.io._
import java.nio.file._

We are now ready to go!

## Shared Elements

Let's pretend to do "good software engineering" and factor common parts rather than copy-pasting stuff all over the place. :)

In [ ]:
val absolute = G(90)
val incremental = G(91)
val mm = G(21)

abstract class Program {

    //start and finish Z
    val clearanceZ = 5.0
    // default feed
    val feed = 100.0
    
    
    def header = Seq(
        mm,
        absolute,
        Empty(F(feed)),
        G(0, Z(clearanceZ))
    )
    
    def body: Seq[Command] //Here you do what you need to do!!
    
    def footer = Seq(
        G(0, Z(clearanceZ)),
        M(2)
    )
    
    def program = header ++ body ++ footer

    def save(fileName: String, printer: (Seq[Command], String) => Unit = Printer.apply) = {
        printer(program, fileName)
    }
    
    def display = Viewer.display(program)
    
    override def toString = {
        val writer = new StringWriter
        Printer(program, new BufferedWriter(writer))
        writer.toString
    }
    
}

## Surfacing

Some simple programs to make flat surfaces.

### Round Surface

Make a spiral from the center to the outside.

In [ ]:
class CicularSurface(cutterDiameter: Double, maxRadius: Double,
                     climb: Boolean = true, insideOut: Boolean = true,
                     _feed: Double = 200, helixDepthPerTurn: Double = 2,
                     stepOver: Double = 0.6, precision: Double = 0.5
                    ) extends Program {
    
    import scala.collection.mutable.ArrayBuffer

    override val feed = _feed
    
    val effectiveCutting = cutterDiameter * stepOver
    val maxCuttingRadius = maxRadius - cutterDiameter / 2
    def r(t: Double, t0: Double = 0.0) = math.min(maxCuttingRadius, effectiveCutting * (1 + (t-t0) / 2 / math.Pi))
    def dir(t: Double) = if (climb) t else -t
    def x(t: Double, t0: Double = 0.0) = r(t, t0) * math.cos(dir(t))
    def y(t: Double, t0: Double = 0.0) = r(t, t0) * math.sin(dir(t))
    //find the next t with for the given precision
    def nextStep(t: Double, t0: Double = 0.0) = precision / r(t, t0)
    
    // Do an helix instead of a straight plunge
    def makeHelix = {
        val cmds = ArrayBuffer.empty[Command]
        val r = if (insideOut) effectiveCutting else maxCuttingRadius
        def z(t: Double) = math.max(0.0, clearanceZ - t * helixDepthPerTurn / 2 / math.Pi)
        def dir(t: Double) = if (insideOut == climb) t else -t
        var t = 0.0
        while (z(t) > 0.0) {
            val x = r * math.cos(dir(t))
            val y = r * math.sin(dir(t))
            cmds += G(1, X(x), Y(y), Z(z(t)))
            t += nextStep(t)
        }
        (cmds.toSeq, t % (2 * math.Pi))
    }
    
    def circle(r: Double, t0: Double) = {
        import scala.math.BigDecimal // because scala 2.13 ...
        val start = BigDecimal(t0)
        val end = BigDecimal(t0 + 2 * math.Pi)
        val step = BigDecimal(precision / r)
        for (t <- start until end by step) yield {
            val x = r * math.cos(dir(t.toDouble))
            val y = r * math.sin(dir(t.toDouble))
            G(1, X(x), Y(y))
        }
    }
    
    def makeSpiralIO(t0: Double = 0.0) = {
        val cmds = ArrayBuffer.empty[Command]
        // first a circle to make sure everything is flat
        cmds ++= circle(effectiveCutting, t0)
        // then a spiral from the center to maxRadius
        var t = t0
        while (r(t, t0) < maxCuttingRadius) {
            cmds += G(1, X(x(t, t0)), Y(y(t, t0)))
            t += nextStep(t, t0)
        }
        val delta = nextStep(t, t0)
        val oldT = t
        // finally a normal circle at maxRadius
        cmds ++= circle(maxCuttingRadius, t)
        cmds.toSeq
    }
    
    def makeSpiralOI(t0: Double = 0.0) = {
        val cmds = ArrayBuffer.empty[Command]
        // first do a normal circle at maxRadius
        cmds ++= circle(maxCuttingRadius, t0).reverse
        val tStartMin = (maxCuttingRadius / effectiveCutting - 1) * 2 * math.Pi // r(t) == maxCuttingRadius
        var t = (t0 - tStartMin) % (2*math.Pi) + 2*math.Pi + tStartMin //first value above tStartMin aligned with t0
        val delta = nextStep(t, t0)
        // then spiral toward the center
        while (r(t, t0) > effectiveCutting) {
            cmds += G(1, X(x(t, t0)), Y(y(t, t0)))
            t -= nextStep(t, t0)
        }
        // finally a circle to make sure everything is flat
        cmds ++= circle(effectiveCutting, t).reverse
        cmds.toSeq
    }
    
    def body = {
        val (helix, t0) = makeHelix
        val spiral = if(insideOut) makeSpiralIO(t0) else makeSpiralOI(t0)
        val (x0, y0) = helix.head match {
            case G(1, _, Seq(X(x0), Y(y0), Z(_))) => (x0, y0)
        }
        Seq(
            G(0, X(x0), Y(y0)), // init position
            Empty(F(feed)), // setNormalFeed
        ) ++ helix ++ spiral
    }
}


In [ ]:
new CicularSurface(6, 38).display
//new CicularSurface(6, 38).save("spiral_in-out_climb.nc")
//new CicularSurface(6, 38, climb = false).save("spiral_in-out_conventional.nc")
//new CicularSurface(6, 38, insideOut = false).save("spiral_out-in_climb.nc")
//new CicularSurface(6, 38, insideOut = false, climb = false).save("spiral_out-in_conventional.nc")

### Rectangle Surface

Lower left corner is `(0,0,0)`.
Start from the outside and move twoard the center.

In [ ]:
class RectangleSurface(cutterDiameter: Double, x: Double, y: Double,
                       xStartMin: Boolean = true, yStartMin: Boolean = true,
                       _feed: Double = 200, plungeFeed: Double = 100,
                       stepOver: Double = 0.9, climb: Boolean = true) extends Program {
    
    override val feed = _feed
    val effectiveCutting = cutterDiameter * stepOver
    val xMid = x/2
    val yMid = y/2
    val xSteps = ((xMid - cutterDiameter) / effectiveCutting).ceil.toInt
    val ySteps = ((yMid - cutterDiameter) / effectiveCutting).ceil.toInt
    
    def round(offset: Double, prevOffset: Option[Double]) = {
        val extra = prevOffset match {
            case Some(po) => ((offset - po) * math.sqrt(2) - cutterDiameter) / math.sqrt(2)
            case None => 0.0
        }
        val o = offset + cutterDiameter/2
        val corners = Array(
            G(1, X(o), Y(o)),                         // lower left
            G(1, X(o - extra), Y(o - extra)),         // lower left extra 
            G(1, X(o), Y(y - o)),                     // upper left
            G(1, X(o - extra), Y(y - o + extra)),     // upper left extra
            G(1, X(x - o), Y(y - o)),                 // upper right
            G(1, X(x - o + extra), Y(y - o + extra)), // upper right extra
            G(1, X(x - o), Y(o)),                     // lower right
            G(1, X(x - o + extra), Y(o - extra))      // lower right extra
        )
        val idxOffset = (xStartMin, yStartMin) match {
            case (true, true) => 0
            case (true, false) => 2
            case (false, true) => 6
            case (false, false) => 4
        }
        val jSeq = if (extra <= 0.0) Seq(0) else Seq(0,1,0) 
        val seq = for (i <- 0 to 4;
                       j <- if (i > 0 && i < 4) jSeq else Seq(0))
                  yield corners( (2*i + j + idxOffset) % 8 )
        if (climb) seq else seq.reverse
    }
    
    def body = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        val xStart = if (xStartMin) cutterDiameter/2  else x - cutterDiameter/2
        val yStart = if (yStartMin) cutterDiameter/2  else y - cutterDiameter/2
        cmds += G(0, X(xStart), Y(yStart)) // init position
        cmds += G(1, Z(0.0), F(plungeFeed)) // plunge
        cmds += Empty(F(feed)) // setNormalFeed
        cmds ++= round(0, None) //make a slot on the external perimeter
        for (i <- 1 to math.min(xSteps, ySteps)) {
            cmds ++= round(i * effectiveCutting, Some((i-1) * effectiveCutting))
        }
        cmds.toSeq
    }
}

In [ ]:
new RectangleSurface(4, 20, 20, stepOver = 0.75).display
//new RectangleSurface(4, 20, 20, stepOver = 0.75).save("rect_test_1.nc")
//new RectangleSurface(4, 20, 20, stepOver = 0.75, xStartMin = false).save("rect_test_2.nc")
//new RectangleSurface(4, 20, 20, stepOver = 0.75, yStartMin = false).save("rect_test_3.nc")
//new RectangleSurface(4, 20, 20, stepOver = 0.75, xStartMin = false, yStartMin = false).save("rect_test_4.nc")

## Helix

In [ ]:
def helix(x: Double, y: Double, z: Double,
          radius: Double, pitch: Double, nbrTurns: Int,
          clockwise: Boolean = true) = {
    val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
    buffer += G(0, X(radius), Y(y), Z(z))
    for (n <- 0 until nbrTurns) yield {
        val dir = if (clockwise) 2 else 3
        buffer += G(dir, X(-radius), I(-radius), Z(z + n*pitch + pitch/2))
        buffer += G(dir, X(radius), I(radius), Z(z + n*pitch + pitch))
    }
    buffer.toSeq
}

In [ ]:
Viewer.display(helix(0, 0, 0, 5, -2, 5))

In [ ]:
Viewer.display(helix(0, 0, 0, 5, 1, 5, false))

## Storage box

A small box to store some grinding stones.

Here is a photo of the result:

![box](./img/box.jpg)

In [ ]:
object Box {
    
    // the stones to store (rectangular)
    val stoneLength = 60
    val stoneWidth = 6
    val stoneHeight = 25
    val numberOfStones = 4
    
    val cutterRadius = 3
    val depthOfPlunge = 4
    val feed = 200
    val plungeFeed = 100
    val clearanceZ = 5
    
    val boxLength = stoneLength + 2*cutterRadius + 8
    val boxWidth = stoneWidth + (numberOfStones-1)*(stoneWidth+4) + 8
    val boxCornerRadius = 4
    val boxHeight = 20
    val lidOverlap = 5
    val lidLip = 2
    val lidHeight = 10
    val stoneSpacing = 10
    val stoneToBorder = 4 + cutterRadius
    
    //machine position not including cutter
    def slot(xStart: Double, xStop: Double,
             yStart: Double, yStop: Double,
             zStart: Double, zStop: Double) = {
        var cmds = Seq( G(0, X(xStart), Y(yStart)) )
        
        def loop(z: Double) = {
            Seq(
                G(1, Z(z), F(plungeFeed)),
                G(1, X(xStop), F(feed)),
                G(1, Y(yStop)),
                G(1, X(xStart)),
                G(1, Y(yStart))
            )
        }
        var z = zStart
        while (z > zStop) {
            cmds = cmds ++ loop(z)
            z = z-depthOfPlunge
        }
        cmds = cmds ++ loop(zStop)
        cmds = cmds :+ G(0, Z(clearanceZ))
        cmds
    }
    
    //       p3_________p4
    //       /          \
    //    p2|            |p5
    //      |            |
    //      |            |
    //      |            |
    //    p1|            |p6
    //       \__________/
    //      p0         p7
    def roundedRectangleOuter(
            xMin: Double, xMax: Double, //actual dimension
            yMin: Double, yMax: Double, //actual dimension
            z: Double, cornerRadius: Double) = {
        val (x0, y0) = (xMin + cornerRadius, yMin - cutterRadius)
        val (x1, y1) = (xMin - cutterRadius, yMin + cornerRadius)
        val (x2, y2) = (xMin - cutterRadius, yMax - cornerRadius)
        val (x3, y3) = (xMin + cornerRadius, yMax + cutterRadius)
        val (x4, y4) = (xMax - cornerRadius, yMax + cutterRadius)
        val (x5, y5) = (xMax + cutterRadius, yMax - cornerRadius)
        val (x6, y6) = (xMax + cutterRadius, yMin + cornerRadius)
        val (x7, y7) = (xMax - cornerRadius, yMin - cutterRadius)
        Seq(
            G(0, X(x0), Y(y0)),
            G(1, Z(z), F(plungeFeed)),
            G(2, X(x1), Y(y1), R(cornerRadius+cutterRadius), F(feed)),
            G(1, X(x2), Y(y2)),
            G(2, X(x3), Y(y3), R(cornerRadius+cutterRadius)),
            G(1, X(x4), Y(y4)),
            G(2, X(x5), Y(y5), R(cornerRadius+cutterRadius)),
            G(1, X(x6), Y(y6)),
            G(2, X(x7), Y(y7), R(cornerRadius+cutterRadius)),
            G(1, X(x0), Y(y0)),
            G(0, Z(clearanceZ))
        )
    }
    
    def roundedRectangleInner(
            xMin: Double, xMax: Double, //actual dimension
            yMin: Double, yMax: Double, //actual dimension
            z: Double, cornerRadius: Double) = {
        if (cornerRadius > cutterRadius) {
            val (x0, y0) = (xMin + cornerRadius, yMin + cutterRadius)
            val (x1, y1) = (xMin + cutterRadius, yMin + cornerRadius)
            val (x2, y2) = (xMin + cutterRadius, yMax - cornerRadius)
            val (x3, y3) = (xMin + cornerRadius, yMax - cutterRadius)
            val (x4, y4) = (xMax - cornerRadius, yMax - cutterRadius)
            val (x5, y5) = (xMax - cutterRadius, yMax - cornerRadius)
            val (x6, y6) = (xMax - cutterRadius, yMin + cornerRadius)
            val (x7, y7) = (xMax - cornerRadius, yMin + cutterRadius)
            Seq(
                G(0, X(x0), Y(y0)),
                G(1, Z(z), F(plungeFeed)),
                G(1, X(x7), Y(y7), F(feed)),
                G(3, X(x6), Y(y6), R(cornerRadius - cutterRadius)),
                G(1, X(x5), Y(y5)),
                G(3, X(x4), Y(y4), R(cornerRadius - cutterRadius)),
                G(1, X(x3), Y(y3)),
                G(3, X(x2), Y(y2), R(cornerRadius - cutterRadius)),
                G(1, X(x1), Y(y1)),
                G(3, X(x0), Y(y0), R(cornerRadius - cutterRadius)),
                G(0, Z(clearanceZ))
            )
        } else {
            val (x0, y0) = (xMin + cutterRadius, yMin + cutterRadius)
            val (x1, y1) = (xMin + cornerRadius, yMax - cutterRadius)
            val (x2, y2) = (xMax - cutterRadius, yMax - cutterRadius)
            val (x3, y3) = (xMax - cornerRadius, yMin + cutterRadius)
            Seq(
                G(0, X(x0), Y(y0)),
                G(1, Z(z), F(plungeFeed)),
                G(1, X(x3), Y(y3), F(feed)),
                G(1, X(x2), Y(y2)),
                G(1, X(x1), Y(y1)),
                G(1, X(x0), Y(y0)),
                G(0, Z(clearanceZ))
            )
        }
    }
    
    def box = {
        var cmds = Seq[Command](G(0, Z(clearanceZ)))
        //careful here: 25 is the max depth for my 6mm endmill wich is the same as stoneHeight
        //removing the stock around the box
        for (depth <- Seq(-stoneHeight/2, -stoneHeight);
             i <- Seq(4, 1)) {
            cmds = cmds ++ roundedRectangleOuter(-i, (numberOfStones+1)*stoneSpacing+i,
                                                 -i, stoneLength + 2*stoneToBorder+i,
                                                 depth,6+i)
        }
        //outer
        cmds = cmds ++ roundedRectangleOuter(0, (numberOfStones+1)*stoneSpacing,
                                             0, stoneLength + 2*stoneToBorder,
                                             -stoneHeight,6)
        // lip
        cmds = cmds ++ roundedRectangleOuter(lidLip, (numberOfStones+1)*stoneSpacing-lidLip,
                                             lidLip, stoneLength + 2*stoneToBorder -lidLip,
                                             -5, 4)
        // slots for the stones
        for (i <- 1 to numberOfStones) {
            cmds = cmds ++ slot(i*stoneSpacing-1, i*stoneSpacing+1,
                                stoneToBorder, stoneToBorder+stoneLength,
                                -depthOfPlunge, -stoneHeight+2)
        }
        cmds
    }
    
    def lid = {
        var cmds = Seq[Command](G(0, Z(clearanceZ)))
        //outer
        cmds = cmds ++ roundedRectangleOuter(0, (numberOfStones+1)*stoneSpacing,
                                             0, stoneLength + 2*stoneToBorder,
                                             -10,6)
        // clear inner up to the lip
        var i = math.min(stoneLength + 2*stoneToBorder - 2*lidLip, (numberOfStones+1)*stoneSpacing-2*lidLip) / 2.0 - cutterRadius* 1.5
        while (i > 0) {
            cmds = cmds ++ roundedRectangleInner(lidLip + i, (numberOfStones+1)*stoneSpacing - lidLip -i,
                                                 lidLip + i, stoneLength + 2*stoneToBorder - lidLip -i,
                                                 -5,4)
            i -= cutterRadius * 1.5 ///0.75 step over
        }
        cmds = cmds ++ roundedRectangleInner(lidLip, (numberOfStones+1)*stoneSpacing-lidLip,
                                             lidLip, stoneLength + 2*stoneToBorder - lidLip,
                                             -5, 4)
        // slots for the stones
        for (i <- 1 to numberOfStones) {
            cmds = cmds ++ slot(i*stoneSpacing-1, i*stoneSpacing+1,
                                stoneToBorder, stoneToBorder+stoneLength,
                                -5-depthOfPlunge, -8)
        }
        cmds
    }
    
}

In [ ]:
Viewer.display(Box.box)

In [ ]:
Viewer.display(Box.lid)

In [ ]:
save(Box.box, "box.nc")
save(Box.lid, "lid.nc")

## Tube adapter for CNC coolant

An adatper that plugs into a lightltly pressurised bottle.
The dimensions need to be somewhat accurate but the sealing is assured by an o-ring.

This is done in two steps:
1. getting the outer shape with a normal endmill,
2. making the grove for the o-ring and retainer clip with a small slitting saw.

The hole in the center of the tube has been drilled before and it is used to find the 0 (top surface, center of the hole).

Here is an image of the part (sorry for the poor image quality):

![coolant_adapter](img/coolant_adapter.jpg)

This is not the nicest part.
The vise was not tight enough so the part did move and it chipped the endmill.
But it is functional.

In [ ]:
class TubeAdapterShape extends Program {
    
    // some info about the tool
    val cutterRadius = 3 // 6mm cutter
    val backlash = 0.1 // cut a bit smaller than the exact dimensions
    val stockRadius = 9 // it is actuall smaller than that but better be safe
    val radialDepthOfCut = 0.5
    val verticalDepthOfCut = 4
    override val feed = 100
    val clearance = 1
    
    // about the piece
    val topRadius = 4
    val bottomRadius = 5
    val topLength = 9
    val bottomLength = 7
    
    
    val safeRadius = stockRadius + cutterRadius + clearance
    
    def myHelix(radius: Double, depth: Double) = {
        val totalDepth = depth + clearance
        val effectiveRadius = radius + cutterRadius
        val nbrTurns = (totalDepth / verticalDepthOfCut).ceil.toInt
        val pitch = totalDepth / nbrTurns
        helix(0, 0, clearance, effectiveRadius, -pitch, nbrTurns) ++ Seq(
            G(2, X(-effectiveRadius), I(-effectiveRadius)),
            G(2, X(effectiveRadius), I(effectiveRadius)),
            G(1, X(safeRadius)),
            G(1, Z(clearance))
        )
    }
    
    def roughing(startRadius: Double, endRadius: Double, length: Double) = {
        val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
        var radius = startRadius
        while(radius > endRadius) {
            radius = math.max(radius - radialDepthOfCut, endRadius)
            buffer ++= myHelix(radius, length)
        }
        buffer.toSeq
    }
    
    def body = {
        // roughing bottom
        roughing(stockRadius, bottomRadius, topLength + bottomLength) ++
        // roughing top
        roughing(bottomRadius, topRadius, topLength) ++
        // finshingBot
        myHelix(bottomRadius - backlash, topLength + bottomLength) ++ //could start lower and save some time ...
        // finishingTop
        myHelix(topRadius - backlash, topLength)
    }
    
}

In [ ]:
new TubeAdapterShape().display

In [ ]:
new TubeAdapterShape().save("TubeAdapterShape.nc")

In [ ]:
class TubeAdapterSlots extends Program {
    
    import scala.collection.mutable.ArrayBuffer
    
    // some info about the tool
    val cutterRadius = 9.5 / 2
    val cutterHeight = 1
    val backlash = 0.1 // cut a bit smaller than the exact dimensions

    val radialDepthOfCut = 0.5
    val zStepOver = 0.6
    override val feed = 100
    val clearance = 1
    
    // about the piece
    val topRadius = 4
    val bottomRadius = 5
    val topSlot = 5.9 / 2
    val topStart = 3
    val topEnd = 5
    val bottomSlot = 4
    val bottomStart = 11
    val bottomEnd = 12.5
        
    val safeRadius = bottomRadius + cutterRadius + clearance
    
    def slot(radiusStart: Double, radiusEnd: Double,
             start: Double, end: Double) = {
        val buffer = ArrayBuffer.empty[Command]
        buffer += G(0, Z(clearance))
        buffer += G(0, X(safeRadius))
        var z = -start
        slit(radiusStart+cutterRadius, radiusEnd+cutterRadius, z, buffer)
        while (z > -end + cutterHeight) {
            z = math.max(z-cutterHeight*zStepOver, -end + cutterHeight)
            slit(radiusStart+cutterRadius, radiusEnd+cutterRadius, z, buffer)
        } 
        buffer += G(0, X(safeRadius))
        buffer += G(0, Z(clearance))
        buffer.toSeq
    }
    
    def slit(radiusStart: Double, radiusEnd: Double, height: Double,
             buffer: ArrayBuffer[Command]) = {
        buffer += G(0, Z(height))
        buffer += G(1, X(radiusStart))
        val nbrTurns = ((radiusStart - radiusEnd) / radialDepthOfCut).ceil.toInt
        val depthPerTurn = (radiusStart - radiusEnd) / nbrTurns
        for (i <- 0 until nbrTurns) {
            val center = if (i % 2 == 0) radialDepthOfCut / 2 else - radialDepthOfCut / 2
            val xStart  = math.max(radiusEnd, radiusStart - i * radialDepthOfCut)
            val xMiddle = math.max(radiusEnd, radiusStart - (i+0.5) * radialDepthOfCut)
            val xEnd    = math.max(radiusEnd, radiusStart - (i+1) * radialDepthOfCut)
            buffer += G(2, X(-xMiddle), I(-(xStart+xMiddle)/2))
            buffer += G(2, X(xEnd), I((xMiddle+xEnd)/2))
        }
        buffer += G(2, X(-radiusEnd), I(-radiusEnd))
        buffer += G(2, X(radiusEnd), I(radiusEnd))
        buffer += G(1, X(safeRadius))
    }
    
    def body = {
        slot(topRadius, topSlot, topStart, topEnd) ++
        slot(bottomRadius, bottomSlot, bottomStart, bottomEnd)
    }
    
}

In [ ]:
new TubeAdapterSlots().display

In [ ]:
new TubeAdapterSlots().save("TubeAdapterSlots.nc")

## Drilling Pilots Holes

I needed to get drill holes accurately.
A CNC is ideal for this job.
Howver, the CNC I have access to does not have enough torque to drill 6mm hole 20mm deep in aluminium.
So this program is just starting some small pilot holes and I'll finish the hole on a drill press.

In [ ]:
abstract class DrillingProgram(_feed: Double,
                               spindleRPM: Int,
                               drillDepth: Double,
                               peckDepth: Double,
                               emulateG83: Boolean = false) extends Program {
    
    import scala.collection.mutable.ArrayBuffer
    
    // parameters
    override val feed = _feed
    val safeZ = 1.0
    val deltaZ = 0.5
    // to specify
    val xs: Seq[Double]
    val ys: Seq[Double]
    val pointIndices: Seq[(Int,Int)]

    override def header = super.header ++ Seq(M(3, S(spindleRPM))) 
    
    override def footer = Seq(M(5)) ++ super.footer
    
    def body = {
        val buffer = ArrayBuffer.empty[Command]
        if (!emulateG83) {
            buffer += G(98)
            buffer += G(83, R(safeZ), Q(peckDepth))
            for ((x,y) <- pointIndices) buffer += Empty( X(xs(x)), Y(ys(y)), Z(drillDepth))
            buffer += G(80)
        } else {
            for ((x,y) <- pointIndices) {
                buffer += G(0, X(xs(x)), Y(ys(y)))
                var currentZ = safeZ
                while (currentZ > drillDepth) {
                    buffer += G(0, Z(currentZ + deltaZ))
                    currentZ = math.max(drillDepth, currentZ - peckDepth)
                    buffer += G(1, Z(currentZ))
                    buffer += G(0, Z(safeZ))
                }
                buffer += G(0, Z(clearanceZ))
            }
        }
        buffer.toSeq
    }
    
}

class PilotsHolesTopPlate(f: Double, s: Int, d: Double, p: Double, e: Boolean = false) extends DrillingProgram(f,s,d,p,e) {
    
    val xs = Seq[Double]( 50.5, 104.5, 130, 170, 195.5, 249.5)
    
    val ys = Seq[Double]( 30, 67, 89, 111, 133, 170 )
    
    val pointIndices = Seq[(Int,Int)](
        0 -> 0, 0 -> 1, 0 -> 4, 0 -> 5,
        1 -> 0, 1 -> 1, 1 -> 4, 1 -> 5,
        2 -> 2, 2 -> 3,
        3 -> 2, 3 -> 3,
        4 -> 0, 4 -> 1, 4 -> 4, 4 -> 5,
        5 -> 0, 5 -> 1, 5 -> 4, 5 -> 5
    )
    
}

class PilotsHolesMiddlePlate(f: Double, s: Int, d: Double, p: Double, e: Boolean = false) extends DrillingProgram(f,s,d,p,e) {
    
    val xs = Seq[Double]( 20, 73, 105,       127, 160,       200, 233,    255,      287,     340,  //1st mounting position
                                       120.5,         174.5, 200,     240,    265.5,    319.5)      //2nd mounting position
    
    val ys = Seq[Double]( 5, 7.5, 42, 47.5, 72.5, 89, 111, 127.5, 152.5, 158, 192.5, 195)
    
    val pointIndices = Seq[(Int,Int)](
        // rails + 1st mounting position
        0 -> 4, 0 -> 7,
        1 -> 0, 1 -> 2, 1 -> 9, 1 -> 11,
        2 -> 1, 2 -> 3, 2 -> 8, 2 -> 10,
        3 -> 0, 3 -> 2, 3 -> 9, 3 -> 11,
        4 -> 5, 4 -> 6,
        5 -> 5, 5 -> 6,
        6 -> 0, 6 -> 2, 6 -> 9, 6 -> 11,
        7 -> 1, 7 -> 3, 7 -> 8, 7 -> 10,
        8 -> 0, 8 -> 2, 8 -> 9, 8 -> 11,
        9 -> 4, 9 -> 7,
        // 2nd mounting position for the bearing block
        10 -> 0, 10 -> 2, 10 -> 9, 10 -> 11,
        11 -> 0, 11 -> 2, 11 -> 9, 11 -> 11,
        12 -> 5, 12 -> 6,
        13 -> 5, 13 -> 6,
        14 -> 0, 14 -> 2, 14 -> 9, 14 -> 11,
        15 -> 0, 15 -> 2, 15 -> 9, 15 -> 11
    )
    
}

In [ ]:
new PilotsHolesTopPlate(160, 7500, -5, 1).display

In [ ]:
new PilotsHolesMiddlePlate(160, 7500, -5, 1).display

In [ ]:
for ((name,feed,rpm,depth,peck) <- Seq(("1mm_5", 200,10000, -5,1),
                                       ("2mm_5", 200, 5000, -5,2),
                                       ("2mm_10",200, 5000,-10,2),
                                       ("3mm_10",200, 5000,-10,4))) {
    new PilotsHolesTopPlate(feed,rpm,depth,peck,true).save("pilot_holes_top_plate_"+name+".nc", Printer.rolandDG)
    new PilotsHolesMiddlePlate(feed,rpm,depth,peck,true).save("pilot_holes_middle_plate_"+name+".nc", Printer.rolandDG)    
}

## Engraving

Test engraving with a V-bit.
That particular one was for a gift.

In [ ]:
class EngravingTest(xyDimension: Double, zDimension: Double,
                    depthOfCut: Double = 0.5, finishingPass: Double = 0.2,
                    offsets: Seq[(Double,Double)] = Seq((0,0))) extends Program {
    
    override val feed = 200
    
    // generated from a bezier curve using blender
    val points_half = Seq(
       ( 0.000000, 11.086214, 0.000000), 
       ( 0.806945, 11.764009,-0.015200),
       ( 1.640551, 12.316256,-0.059065),
       ( 2.493831, 12.747746,-0.128987),
       ( 3.360688, 13.063274,-0.222361),
       ( 4.235025, 13.267637,-0.336582),
       ( 5.110744, 13.365633,-0.469043),
       ( 5.981749, 13.362055,-0.617139),
       ( 6.841942, 13.261698,-0.778264),
       ( 7.685225, 13.069359,-0.949813),
       ( 8.505503, 12.789831,-1.129178),
       ( 9.296677, 12.427912,-1.313756),
       (10.052650, 11.988398,-1.500939),
       (10.767326, 11.476082,-1.688122),
       (11.434607, 10.895760,-1.872699),
       (12.048395, 10.252230,-2.052064),
       (12.602596,  9.550283,-2.223613),
       (13.091107,  8.794720,-2.384738),
       (13.507836,  7.990332,-2.532834),
       (13.846685,  7.141917,-2.665295),
       (14.101556,  6.254269,-2.779516),
       (14.266351,  5.332184,-2.872890),
       (14.334973,  4.380458,-2.942812),
       (14.301327,  3.403887,-2.986676),
       (14.159315,  2.407268,-3.001877),
       (13.905559,  1.427261,-2.986677),
       (13.547565,  0.495462,-2.942812),
       (13.095561, -0.390940,-2.872890),
       (12.559774, -1.234758,-2.779516),
       (11.950432, -2.038803,-2.665295),
       (11.277760, -2.805887,-2.532834),
       (10.551985, -3.538822,-2.384738),
       ( 9.783336, -4.240419,-2.223613),
       ( 8.982038, -4.913491,-2.052064),
       ( 8.158319, -5.560850,-1.872699),
       ( 7.322405, -6.185307,-1.688122),
       ( 6.484524, -6.789674,-1.500939),
       ( 5.654902, -7.376763,-1.313756),
       ( 4.843767, -7.949385,-1.129178),
       ( 4.061346, -8.510353,-0.949813),
       ( 3.317865, -9.062479,-0.778264),
       ( 2.623551, -9.608573,-0.617139),
       ( 1.988631,-10.151449,-0.469043),
       ( 1.423333,-10.693916,-0.336582),
       ( 0.937883,-11.238791,-0.222361),
       ( 0.542508,-11.788879,-0.128987),
       ( 0.247436,-12.346998,-0.059065),
       ( 0.062892,-12.915956,-0.015201),
       ( 0.000000,-13.498569, 0.000000)
    )
    
    def range(values: Seq[Double]) = values.max - values.min
    
    val points = {
        val p0 = points_half ++ points_half.reverse.tail.map{ case (x,y,z) => (-x,y,z) }
        val scalingXY = xyDimension / math.max(range(p0.map(_._1)),range(p0.map(_._2)))
        val scalingZ = zDimension / range(p0.map(_._3))
        p0.map{ case (x,y,z) => (scalingXY * x, scalingXY * y, scalingZ * z) }
    }

    def layer(xOffset: Double, yOffset: Double, zOffset: Double) = {
        points.map{ case (x,y,z) => G(1, X(x + xOffset), Y(y + yOffset), Z(z + zOffset)) }
    }
    
    def instance(xOffset: Double, yOffset: Double) = {
        val initPos = G(0, X(points.head._1 + xOffset), Y(points.head._2 + yOffset))
        var currentZ = zDimension
        var passes: List[Double] = Nil
        while (currentZ > finishingPass) {
            currentZ = math.max(currentZ - depthOfCut, finishingPass)
            passes = currentZ :: passes
        }
        passes = 0.0 :: passes
        initPos +: passes.reverse.flatMap(layer(xOffset, yOffset, _)) :+ G(0, Z(clearanceZ))
    }
    
    def body = {
        offsets.flatMap({ case (x,y) => instance(x,y) })
    }
   
}

In [ ]:
new EngravingTest(30, 3).display
//new EngravingTest().save("engraving_test.nc")

In [ ]:
new EngravingTest(30, 3, offsets = Seq((23,23), (23,67), (67,23), (67,67))).display
//new EngravingTest().save("engraving_test.nc")

## Misc

### Modifying a program using pattern matching

We take a spiral and instead of keeping it flat, we add a small curve on the Z axis.

_Remark._ A cleaner version should use a `Transducer`.

In [ ]:
class CurvedSurface(cutterDiameter: Double, maxRadius: Double, minZ: Double,
                    climb: Boolean = true, insideOut: Boolean = true,
                    _feed: Double = 200, helixDepthPerTurn: Double = 2,
                    stepOver: Double = 0.6, precision: Double = 0.5,
                    ) extends CicularSurface(cutterDiameter, maxRadius, climb, insideOut,
                                             _feed, helixDepthPerTurn, stepOver, precision) {
    override def body = {
        val cmds = super.body
        cmds.map( f => f match {
            case G(1, _, Seq(X(x), Y(y))) =>
                val r = math.hypot(x, y) / maxRadius
                val z = minZ * r*r
                G(1, X(x), Y(y), Z(z))
            case other => other
        })
    }
}

new CurvedSurface(6, 38, -3, stepOver = 0.5).save("hyperbol_rough.nc")
new CurvedSurface(3.2, 36.6, -5, stepOver = 0.2).save("hyperbol.nc")

### making a jig to attach some parts

In [ ]:
object Jig1 {

    val n = 4
    val dy = 15.0
    val toKeep = 5.0
    val slotDepth = 3.5
    val slotWidth = 8.0
    val drillDepth = Z(-10)

    val cutterDiam = 6.0
    val stepOver = 0.9
    val feedMetal = F(120)
    val feedWood = F(200)
    val feedZ = F(100)
    val clearZ = Z(5)
    
    val yMin = Y(-10 - cutterDiam)
    val yMax = Y((n-1)*dy + 10 + cutterDiam)
    def nextY(i: Int) = if (i % 2 == 0) yMax else yMin
    
    protected def buffer = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        cmds += mm
        cmds += absolute
        cmds += G(1, clearZ, feedZ)
        cmds
    }
    
    def slot = {
        val cmds = buffer
        val passes = ( slotWidth / (cutterDiam * stepOver) ).ceil.toInt
        assert(passes > 1 && slotWidth > cutterDiam)
        val effectiveSlot = slotWidth - cutterDiam
        cmds += G(0, X(-effectiveSlot/2), yMin)
        cmds += G(1, Z(-slotDepth), feedZ)    
        for (i <- 0 until passes) {
            cmds += G(1, X(-effectiveSlot/2 + i * effectiveSlot / (passes-1)), feedWood)
            cmds += G(1, nextY(i))
        }
        cmds += G(0, X(0), Y(0))
        cmds += M(2)
        cmds.toSeq
    }
    
    def drill = {
        val cmds = buffer
        cmds += G(0, X(0), Y(0))
        for (i <- 0 until 4) {
            cmds += G(0, Y(i * dy))
            cmds += G(1, drillDepth)
            cmds += G(1, clearZ)
        }
        cmds += G(0, X(0), Y(0))
        cmds += M(2)
        cmds.toSeq
    }
    
    def cut = {
        val dx = (toKeep + cutterDiam) / 2
        val cmds = buffer
        cmds += G(0, X(-dx), yMin)
        cmds += G(1, Z(0), feedZ)
        cmds += G(1, yMax, feedMetal)
        cmds += G(0, X(dx))
        cmds += G(1, yMin)
        cmds += G(1, clearZ, feedZ)
        cmds += M(2)
        cmds.toSeq
    }
    
    def save = {
        Printer(slot, "jig1_slot.nc")
        Printer(drill, "jig1_drill.nc")
        Printer(cut, "jig1_cut.nc")
    }
}

In [ ]:
Jig1.save

## Grinding

A grinding job.
So it moves very slowly along one axis.

In [ ]:
def nbrSteps(start: Double, end: Double, stepSize: Double): Int = ((end - start) / stepSize).abs.floor.toInt
def direction(start: Double, end: Double) = (end - start).sign

val stepSize = 0.01
val yMin = -10.0
val yMax =  10.0

class GrindingZ extends Program {

    val zStart    = 0.0
    val zEnd      = -3.0

    def layer(zIdx: Int) = Seq(
        G(1, Z(zStart + zIdx * stepSize * direction(zStart, zEnd))),
        G(1, Y(yMax)),
        G(0, X(1.0)),
        G(0, Y(yMin)),
        G(0, X(0.0))
    )

    def body = for (z <- 0 to nbrSteps(zStart, zEnd, stepSize); cmd <- layer(z)) yield cmd

}

//this time move alone the x axis
class GrindingX extends Program {

    val xStart    = 0.0
    val xEnd      = -3.5

    def layerX(xIdx: Int) = Seq(
        G(1, X(xStart + xIdx * stepSize * direction(xStart, xEnd).sign)),
        G(1, Y(yMax)),
        G(1, Y(yMin))
    )

    val body = {
        val initPos = Seq(
            G(0, X(0), Y(yMin)),
            G(1, Z(0)))
        val layers = for (i <- 0 to nbrSteps(xStart, xEnd, stepSize); cmd <- layerX(i)) yield cmd
        initPos ++ layers
    }

}

new GrindingZ().save("grinding_z.nc")
new GrindingX().save("grinding_x.nc")